<b>Note: This Jupyter Notebook is associated with the article [The Long Straddle Explained (and How to Trade The Options Strategy with Alpaca)](https://alpaca.markets/learn/long-straddle).</b>

# Step 1: Setting Up the Environment and Trade Parameters

In [ ]:
# Install or upgrade the package `alpaca-py` and import it
!python3 -m pip install --upgrade alpaca-py

import pandas as pd
import numpy as np
from scipy.stats import norm
import alpaca
import time
from scipy.optimize import brentq
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit

from alpaca.data.historical.option import OptionHistoricalDataClient
from alpaca.data.historical.stock import StockHistoricalDataClient, StockLatestTradeRequest
from alpaca.data.requests import StockBarsRequest, OptionLatestQuoteRequest, OptionChainRequest, OptionBarsRequest
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import (
    MarketOrderRequest,
    LimitOrderRequest,
    GetOptionContractsRequest,
    MarketOrderRequest,
    OptionLegRequest,
    ClosePositionRequest,
    CreateWatchlistRequest
)
from alpaca.trading.enums import (
    AssetStatus,
    ExerciseStyle,
    OrderSide,
    OrderClass,
    OrderStatus,
    OrderType,
    TimeInForce,
    QueryOrderStatus,
    ContractType,
)

In [ ]:
# API_KEY = "Alpaca's Trading API Key (Paper Account)"
# API_SECRET = "Alpaca's Trading API Secret Key (Paper Account)"

# A safe approach to setting up API credentials for Alpaca (Assume you run this notebook in Google Colab)
# Add your key to Colab Secrets. Add your API key to the Colab Secrets manager to securely store it
from google.colab import userdata
API_KEY = userdata.get('ALPACA_API_KEY')
API_SECRET = userdata.get('ALPACA_SECRET_KEY')
BASE_URL = None
## We use paper environment for this example (Please do not modify this. This example is for paper trading only).
PAPER = True

# Initialize Alpaca clients
trade_client = TradingClient(api_key=API_KEY, secret_key=API_SECRET, paper=PAPER, url_override=BASE_URL)
option_historical_data_client = OptionHistoricalDataClient(api_key=API_KEY, secret_key=API_SECRET, url_override=BASE_URL)
stock_data_client = StockHistoricalDataClient(api_key=API_KEY, secret_key=API_SECRET)

# Below are the variables for development this documents
# Please do not change these variables
trade_api_url = None
trade_api_wss = None
data_api_url = None
option_stream_data_wss = None

# Step 2: Calculations for Implied Volatility and Option Greeks with Black-Scholes model

In [ ]:
# Calculate implied volatility
def calculate_implied_volatility(option_price, S, K, T, r, option_type):
    
    # Define a reasonable range for sigma
    sigma_lower = 1e-6
    sigma_upper = 5.0  # Adjust upper limit if necessary

    # Check if the option is out-of-the-money and price is close to zero
    intrinsic_value = max(0, (S - K) if option_type == 'call' else (K - S))
    if option_price <= intrinsic_value + 1e-6:
        
        print('Option price is close to intrinsic value; implied volatility is near zero.')
        return 0.0
    
    # Define the function to find the root
    def option_price_diff(sigma):
        d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)
        if option_type == 'call':
            price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
        elif option_type == 'put':
            price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
        return price - option_price

    try:
        return brentq(option_price_diff, sigma_lower, sigma_upper)
    except ValueError as e:
        print(f"Failed to find implied volatility: {e}")
        return None


def calculate_greeks(option_price, strike_price, expiration, underlying_price, risk_free_rate, option_type):
    T = (expiration - pd.Timestamp.now()).days / 365 # It is unconventional, but some use 225 days (# of annual trading days) in replace of 365 days
    T = max(T, 1e-6)  # Set minimum T to avoid zero
    
    if T == 1e-6:
        print('Option has expired or is expiring now; setting Greeks based on intrinsic value.')
        if option_type == 'put':
            delta = -1.0 if underlying_price < strike_price else 0.0
        else:
            delta = 1.0 if underlying_price > strike_price else 0.0
        gamma = 0.0
        theta = 0.0
        vega = 0.0
        return delta, gamma, theta, vega
    
    # Calculate IV
    IV = calculate_implied_volatility(option_price, underlying_price, strike_price, T, risk_free_rate, option_type)

    if IV is None or IV == 0.0:
        print('Implied volatility could not be determined, skipping Greek calculations.')
        return None
    
    d1 = (np.log(underlying_price / strike_price) + (risk_free_rate + 0.5 * IV ** 2) * T) / (IV * np.sqrt(T))
    d2 = d1 - IV * np.sqrt(T) # d2 for Theta calculation
    # Calculate Delta
    delta = norm.cdf(d1) if option_type == 'call' else -norm.cdf(-d1)
    # Calculate Gamma
    gamma = norm.pdf(d1) / (underlying_price * IV * np.sqrt(T))
    # Calculate Vega
    vega = underlying_price * np.sqrt(T) * norm.pdf(d1)
    # Calculate Theta
    if option_type == 'call':
        theta = (
            - (underlying_price * norm.pdf(d1) * IV) / (2 * np.sqrt(T))
            - (risk_free_rate * strike_price * np.exp(-risk_free_rate * T) * norm.cdf(d2))
        )
    else:
        theta = (
            - (underlying_price * norm.pdf(d1) * IV) / (2 * np.sqrt(T))
            + (risk_free_rate * strike_price * np.exp(-risk_free_rate * T) * norm.cdf(-d2))
        )
    # Convert annualized theta to daily theta
    theta /= 365
    
    return delta, gamma, theta, vega

# Step 3: Choosing the Right Strike Price and Expiration and Many Other Factors

In [ ]:
# Select the underlying stock
underlying_symbol = 'AAPL'

# Set the timezone
timezone = ZoneInfo('America/New_York')

# Get current date in US/Eastern timezone
today = datetime.now(timezone).date()

# Define a 5% range around the underlying price 
STRIKE_RANGE = 0.05

# Buying power percentage to use for the trade
BUY_POWER_LIMIT = 0.05

# Risk free rate for the options greeks and IV calculations
risk_free_rate = 0.01

# Check account buying power
buying_power = float(trade_client.get_account().buying_power)

# Set the open interest volume threshold
OI_THRESHOLD = 100

# Calculate the limit amount of buying power to use for the trade
buying_power_limit = buying_power * BUY_POWER_LIMIT

# Set the expiration date range for the options
min_expiration = today + timedelta(days=14)
max_expiration = today + timedelta(days=60)

# Get the latest price of the underlying stock
def get_underlying_price(symbol):
    # Get the latest trade for the underlying stock
    underlying_trade_request = StockLatestTradeRequest(symbol_or_symbols=symbol)
    underlying_trade_response = stock_data_client.get_stock_latest_trade(underlying_trade_request)
    return underlying_trade_response[symbol].price

# Get the latest price of the underlying stock
underlying_price = get_underlying_price(underlying_symbol)

# Set the minimum and maximum strike prices based on the underlying price
min_strike = str(underlying_price * (1 - STRIKE_RANGE))
max_strike = str(underlying_price * (1 + STRIKE_RANGE))

# Define a common expiration range for all legs between 14 days and 42 days
COMMON_EXPIRATION_RANGE = (14, 42)

# Each key corresponds to a leg and maps to a tuple of: (expiration range, IV range, delta range, theta range)
criteria = {
    'long_put':  (COMMON_EXPIRATION_RANGE, (0.20, 0.60), (-0.60, -0.10), (-0.20, -0.005)),
    'long_call': (COMMON_EXPIRATION_RANGE, (0.20, 0.60), (0.10, 0.60), (-0.20, -0.005))
}

# Set target profit levels
TARGET_PROFIT_PERCENTAGE = 0.4

# Display the values
print(f"Underlying Symbol: {underlying_symbol}")
print(f"{underlying_symbol} price: {underlying_price}")
print(f"Strike Range: {STRIKE_RANGE}")
print(f"Buying Power Limit Percentage: {BUY_POWER_LIMIT}")
print(f"Risk Free Rate: {risk_free_rate}")
print(f"Account Buying Power: {buying_power}")
print(f"Buying Power Limit: {buying_power_limit}")
print(f"Open Interest Threshold: {OI_THRESHOLD}")
print(f"Minimum Expiration Date: {min_expiration}")
print(f"Maximum Expiration Date: {max_expiration}")
print(f"Minimum Strike Price: {min_strike}")
print(f"Maximum Strike Price: {max_strike}")


# Step 4: Historical Market Data Analysis with its stock chart

## Stock Price Trends (Bar Chart Analysis)

In [ ]:
# Get the historical data for the underlying stock by symbol and timeframe
# ref. https://alpaca.markets/sdks/python/api_reference/data/option/historical.html
def get_stock_data(underlying_symbol, days=90):
    today = datetime.now(timezone).date()
    req = StockBarsRequest(
        symbol_or_symbols=[underlying_symbol],
        timeframe=TimeFrame(amount=1, unit=TimeFrameUnit.Day),     # specify timeframe
        start=today - timedelta(days=days),                          # specify start datetime, default=the beginning of the current day.
    )
    return stock_data_client.get_stock_bars(req).df

# List of stock agg objects while dropping the symbol column
priceData = get_stock_data(underlying_symbol, days=180).reset_index(level='symbol', drop=True)

import plotly.graph_objects as go

# Bar chart for the stock price
fig = go.Figure(data=[go.Candlestick(x=priceData.index,
                open=priceData['open'],
                high=priceData['high'],
                low=priceData['low'],
                close=priceData['close'])])

fig.show()


## Relative Volatility Index (RVI) for Volatility Directions and Its Degree

In [ ]:
def calculate_rvi(df, period):
    # Calculate daily price changes 
    df['price_change'] = df['close'].diff()

    # Separate up and down price changes
    df['up_change'] = df['price_change'].where(df['price_change'] > 0, 0)
    df['down_change'] = -df['price_change'].where(df['price_change'] < 0, 0)

    # Calculate std of up and down changes over the rolling period
    df['avg_up_std'] = df['up_change'].rolling(window=period).std()
    df['avg_down_std'] = df['down_change'].rolling(window=period).std()

    # Calculate RVI
    df['rvi'] = 100 - (100 / (1 + (df['avg_up_std'] / df['avg_down_std'])))

    # Smooth the RVI with a 4 periods moving average
    df['rvi_smoothed'] = df['rvi'].rolling(window=4).mean()

    return df['rvi_smoothed']

# smoothed RVI
calculate_rvi(priceData, period=21).iloc[30:]

## Bollinger Bands for Volatility Assessment

In [ ]:
# setup bollinger band calculations
def check_bb(df, period=14, multiplier=2):
    bollinger_bands = []
    # Calculate the Simple Moving Average (SMA)
    df['SMA'] = df['close'].rolling(window=period).mean()
    # Calculate the rolling standard deviation
    df['StdDev'] = df['close'].rolling(window=period).std()
    # Calculate the Upper Bollinger Band (two standard deviation)
    df['Upper Band'] = df['SMA'] + (multiplier * df['StdDev'])
    # Calculate the Lower Bollinger Band (two standard deviation)
    df['Lower Band'] = df['SMA'] - (multiplier * df['StdDev'])
    # Get the most recent Upper Band value
    upper_bollinger_band = df['Upper Band'].iloc[-1]
    lower_bollinger_band = df['Lower Band'].iloc[-1]
    
    bollinger_bands = [upper_bollinger_band, lower_bollinger_band]
    return bollinger_bands

bollinger_bands = check_bb(priceData, 14, 2)

# The current market price is not too close to the two-standard deviation level yet but is relatively closer to the higher Bollinger Band.
print(f"Latest Upper Bollinger Band is: {bollinger_bands[0]}. Latest Lower Bollinger Band is {bollinger_bands[1]}; while underlying stock '{underlying_symbol}' price is {underlying_price}.")

# Step 5: Setting Up For A Long Straddle Options Strategy (Find both put and call options)

In [ ]:
# option_type: ContractType.CALL or ContractType.PUT.
def get_options(underlying_symbol, min_strike, max_strike, min_expiration, max_expiration, option_type):
    req = GetOptionContractsRequest(
        underlying_symbols=[underlying_symbol],
        status=AssetStatus.ACTIVE,
        type=option_type,
        strike_price_gte=min_strike,
        strike_price_lte=max_strike,
        expiration_date_gte=min_expiration,
        expiration_date_lte=max_expiration,        
    )
    return trade_client.get_option_contracts(req).option_contracts


In [ ]:
def validate_sufficient_OI(option_data, OI_THRESHOLD):
    '''Ensure that the option has the required fields and sufficient open interest.'''
    if option_data.open_interest is None or option_data.open_interest_date is None:
        return False
    if float(option_data.open_interest) <= OI_THRESHOLD:
        return False
    return True

In [ ]:
def build_option_dict(option_data, underlying_price, risk_free_rate):
    '''
    Helper to build an option dictionary from option_data and calculated metrics.
    '''
    # Retrieve the latest quote
    option_quote_req = OptionLatestQuoteRequest(symbol_or_symbols=option_data.symbol)
    option_quote = option_historical_data_client.get_option_latest_quote(option_quote_req)[option_data.symbol]
    option_price = (option_quote.bid_price + option_quote.ask_price) / 2

    strike_price = float(option_data.strike_price)
    expiration = pd.Timestamp(option_data.expiration_date)
    remaining_days = (expiration - pd.Timestamp.now()).days

    iv = calculate_implied_volatility(
        option_price=option_price,
        S=underlying_price,
        K=strike_price,
        T=max(remaining_days / 365, 1e-6),
        r=risk_free_rate,
        option_type=option_data.type.value
    )
    delta, _, theta, _ = calculate_greeks(
        option_price=option_price,
        strike_price=strike_price,
        expiration=expiration,
        underlying_price=underlying_price,
        risk_free_rate=risk_free_rate,
        option_type=option_data.type.value
    )

    # Build a candidate dictionary that can be augmented or returned
    candidate = {
        'id': option_data.id,
        'name': option_data.name,
        'symbol': option_data.symbol,
        'strike_price': option_data.strike_price,
        'root_symbol': option_data.root_symbol,
        'underlying_symbol': option_data.underlying_symbol,
        'underlying_asset_id': option_data.underlying_asset_id,
        'close_price': option_data.close_price,
        'close_price_date': option_data.close_price_date,
        'expiration': expiration,
        'remaining_days': remaining_days,
        'open_interest': option_data.open_interest,
        'open_interest_date': option_data.open_interest_date,
        'size': option_data.size,
        'status': option_data.status,
        'style': option_data.style,
        'tradable': option_data.tradable,
        'type': option_data.type,
        'initial_IV': iv,
        'initial_delta': delta,
        'initial_theta': theta,
        'initial_option_price': option_price,

    }
    # print(f"candidate{candidate}") # Comment out for further investigation of the condition.
    return candidate

In [ ]:
def check_candidate_option_conditions(candidate, criteria, label):
    '''
    Check whether a candidate option meets the filtering criteria.
    The criteria is a tuple of (expiration_range, iv_range, delta_range, theta_range).
    '''
    expiration_range, iv_range, delta_range, theta_range = criteria

    if not (expiration_range[0] <= candidate['remaining_days'] <= expiration_range[1]):
        print(f"{candidate['symbol']} fails expiration condition for {label}.")
        return False
    if not (iv_range[0] <= candidate['initial_IV'] <= iv_range[1]):
        print(f"{candidate['symbol']} fails IV condition for {label}.")
        return False
    if not (delta_range[0] <= candidate['initial_delta'] <= delta_range[1]):
        print(f"{candidate['symbol']} fails delta condition for {label}.")
        return False
    if not (theta_range[0] <= candidate['initial_theta'] <= theta_range[1]):
        print(f"{candidate['symbol']} fails theta condition for {label}.")
        return False

    return True

In [ ]:
def group_and_select_common_expiration(put_options, call_options, underlying_price, risk_free_rate, criteria, OI_THRESHOLD):
    """
    Processes put and call options, groups valid candidates by expiration, and selects a common expiration.
    
    Returns:
      selected_expiration: The chosen expiration date.
      put_candidates: List of long put candidates for that expiration.
      call_candidates: List of long call candidates for that expiration.
    """
    put_candidates_by_exp = {}
    call_candidates_by_exp = {}

    # Process put options
    # Iterate through each put option, validate its open interest, build its candidate dictionary, and group the candidate by its expiration date. 
    for option_data in put_options:
        if not validate_sufficient_OI(option_data, OI_THRESHOLD):
            print(f"Insufficient open interest for put option {getattr(option_data, 'symbol', 'unknown')} (threshold: {OI_THRESHOLD}). Skipping candidate.")
            continue
        candidate = build_option_dict(option_data, underlying_price, risk_free_rate)
        expiration = candidate['expiration']
        put_candidates_by_exp.setdefault(expiration, [])
        # Check if the candidate meets the long put criteria and add it to the appropriate expiration group.
        if check_candidate_option_conditions(candidate, criteria['long_put'], 'long_put'):
            put_candidates_by_exp[expiration].append(candidate)
            print(f"Added {candidate['symbol']} as a long put candidate for expiration {expiration}.")

    # Process call options
    # Iterate through each call option, validate its open interest, build its candidate dictionary, and group the candidate by its expiration date. 
    for option_data in call_options:
        if not validate_sufficient_OI(option_data, OI_THRESHOLD):
            print(f"Insufficient open interest for put option {getattr(option_data, 'symbol', 'unknown')} (threshold: {OI_THRESHOLD}). Skipping candidate.")
            continue
        candidate = build_option_dict(option_data, underlying_price, risk_free_rate)
        expiration = candidate['expiration']
        call_candidates_by_exp.setdefault(expiration, [])
        # Check if the candidate meets the long call criteria and add it to the appropriate expiration group.
        if check_candidate_option_conditions(candidate, criteria['long_call'], 'long_call'):
            call_candidates_by_exp[expiration].append(candidate)
            print(f"Added {candidate['symbol']} as a long call candidate for expiration {expiration}.")

    # Find common expiration dates
    common_expirations = set(put_candidates_by_exp.keys()).intersection(set(call_candidates_by_exp.keys()))
    if not common_expirations:
        raise Exception('No common expiration found across put and call candidates.')

    # Choose the expiration with the most candidates
    selected_expiration = max(common_expirations, key=lambda exp: len(put_candidates_by_exp[exp]) + len(call_candidates_by_exp[exp]))
    print(f"Selected common expiration: {selected_expiration}")
    
    return put_candidates_by_exp[selected_expiration], call_candidates_by_exp[selected_expiration]

In [ ]:
def select_common_strike_candidate(put_candidates, call_candidates, underlying_price):
    """
    Finds common strike prices between put and call candidates, selects the one closest to the underlying price,
    and returns one candidate from each side with that strike.
    
    Returns:
      selected_strike: The chosen strike price.
      chosen_put: A long put candidate with the selected strike.
      chosen_call: A long call candidate with the selected strike.
    """
    put_strikes = {float(cand['strike_price']) for cand in put_candidates}
    call_strikes = {float(cand['strike_price']) for cand in call_candidates}
    common_strikes = put_strikes.intersection(call_strikes)
    if not common_strikes:
        raise Exception("No common strike price found across put and call candidates for the selected expiration.")
    
    selected_strike = min(common_strikes, key=lambda strike: abs(strike - underlying_price))
    print(f"Selected common strike price: {selected_strike}")
    
    chosen_put = next((cand for cand in put_candidates if float(cand['strike_price']) == selected_strike), None)
    chosen_call = next((cand for cand in call_candidates if float(cand['strike_price']) == selected_strike), None)
    
    if not chosen_put or not chosen_call:
        raise Exception("Could not find valid candidate(s) with the selected strike price.")
    
    return chosen_put, chosen_call

In [ ]:
def check_buying_power(put_option, call_option, buying_power_limit):
    """
    Calculates the total premium paid (risk) for the long straddle and checks it against the buying power limit.
    If the buying power requirement is not met, the exception is thrown and the rest of the code is never executed.
    """
    option_size = float(put_option['size'])
    risk = (put_option['initial_option_price'] + call_option['initial_option_price']) * option_size
    print(f"Calculated long straddle risk: {risk}.")
    
    if risk >= buying_power_limit:
        raise Exception('Buying power limit exceeded for the long straddle risk.')

In [ ]:
def find_options_for_long_straddle(call_options, put_options, underlying_price, risk_free_rate, buying_power_limit, criteria, OI_THRESHOLD):
    """
    Orchestrates the workflow to build a long straddle with a common expiration and common strike after checking the filtering criteria.
    
    Returns a list of legs in the order: [long_put, long_call]
    """
    # Group candidates by expiration and select the best common expiration
    put_candidates, call_candidates = group_and_select_common_expiration(
            put_options, 
            call_options, 
            underlying_price, 
            risk_free_rate, 
            criteria, 
            OI_THRESHOLD
    )
    
    # Within the selected expiration, choose the common strike and corresponding candidates
    chosen_long_put, chosen_long_call = select_common_strike_candidate(
        put_candidates, 
        call_candidates, 
        underlying_price
    )
    
    # Check buying power requirements
    check_buying_power(chosen_long_put, chosen_long_call, buying_power_limit)
    
    # Return the final long straddle legs
    long_straddle = [chosen_long_put, chosen_long_call]
    print('\nReturning long straddle legs:')

    for leg in long_straddle:
        print(f"{leg['symbol']} at strike {leg['strike_price']}")
    
    return long_straddle

In [ ]:
call_options = get_options(underlying_symbol, min_strike, max_strike, min_expiration, max_expiration, ContractType.CALL)
put_options = get_options(underlying_symbol, min_strike, max_strike, min_expiration, max_expiration, ContractType.PUT)

In [ ]:
long_straddle_order_legs = find_options_for_long_straddle(call_options, put_options, underlying_price, risk_free_rate, buying_power_limit, criteria, OI_THRESHOLD)

# Step 6: Executing A Long Straddle Options Strategy

In [ ]:
## Place orders for the long straddle spread if all options are found
if long_straddle_order_legs:
    # Create a list for the order request
    order_legs = []
    ## Append long put
    order_legs.append(OptionLegRequest(
        symbol=long_straddle_order_legs[0]["symbol"],
        side=OrderSide.BUY,
        ratio_qty=1
    ))
    ## Append long call
    order_legs.append(OptionLegRequest(
        symbol=long_straddle_order_legs[1]["symbol"],
        side=OrderSide.BUY,
        ratio_qty=1
    ))

    # Place the order for both legs simultaneously
    req = MarketOrderRequest(
        qty=1,
        order_class=OrderClass.MLEG,
        time_in_force=TimeInForce.DAY,
        legs=order_legs
    )
    res = trade_client.submit_order(req)
    print("Long Straddle order placed successfully.")

# Cancel Multi-leg Orders (Optional)

You can cancel the order only when the order has not be filled.

In [ ]:
# Query by the order's id
q1 = trade_client.get_order_by_client_id(res.client_order_id)

# Replace overall order
if q1.status != OrderStatus.FILLED:
    # Cancel the whole order
    trade_client.cancel_order_by_id(res.id)
    print(f"Canceled order: {res}")

else:
    print("Order is already filled.")

In [ ]:
# Initialize the Option Historical Data Client
option_historical_data_client = OptionHistoricalDataClient(
    api_key=API_KEY, 
    secret_key=API_SECRET, 
    url_override=BASE_URL
)

# Define the request parameters
req = OptionBarsRequest(
    #  iron_condor_order_legs[1]["symbol"] = the short put option
    symbol_or_symbols=long_straddle_order_legs[1]["symbol"],
    timeframe=TimeFrame.Day,  # Choose timeframe (Minute, Hour, Day, etc.)
    start="2025-01-01",  # Start date
    end="2025-02-21"  # End date
)

option_historical_data_client.get_option_bars(req)


# Backtesting

In [ ]:
# Initialize the Option Historical Data Client
option_historical_data_client = OptionHistoricalDataClient(
    api_key=API_KEY, 
    secret_key=API_SECRET, 
    url_override=BASE_URL
)

# Define the request parameters
req = OptionBarsRequest(
    #  long_straddle_order_legs[0]["symbol"] = the long put option
    symbol_or_symbols=long_straddle_order_legs[0]["symbol"],
    timeframe=TimeFrame.Day,  # Choose timeframe (Minute, Hour, Day, etc.)
    start="2025-01-01",  # Start date
    end="2025-03-14"  # End date
)

option_historical_data_client.get_option_bars(req)
